In [6]:
from json import load
import numpy as np
from neuralNetwork import NeuralNetwork
import torch
import torch.nn as nn
from random import shuffle, randint
from transformers import RobertaModel, RobertaTokenizer
from helper import dict_lists_to_list_of_dicts, get_dataloader
from sklearn.metrics import accuracy_score, f1_score

In [2]:
f = open("ECHR_Corpus.json")
dataset = load(f)
f.close()

# Dataset
ogni caso nel dataset è organizzato così:
```json
{
    "text":"il testo completo della sentenza"
    "clauses":[
        {
            "_id": "id della clause",
            "start": "index di inizio della clause",
            "end": "index di fine della cluause"
        },
        {
            "...":"..."
        }
    ]
    "argument":[
        {
            "premises":[
                "id premise1", "id premise2", "..."
            ],
            "conclusion": "id conclusion"
        },
        {
            "...":"..."
        }
    ]
}
```
In questo passaggio riorganizzo i dati per avere comunque la stessa struttura ma trasformo le clauses in un dizionario che ha come id la chiave della clause e come valore il testo (senza quindi dover usare start e end per cercarlo nel testo). Non tutte le clause sono parte di una premessa o di una conclusione.

In [3]:
refactored_dataset = []
for datapoint in dataset:
    text = datapoint["text"]
    dict_clauses = {}
    for clause in datapoint["clauses"]:
        start = clause["start"]
        end = clause["end"]
        id = clause["_id"]
        dict_clauses[id] = text[start:end]
    refactored_dataset.append({
        "text": text,
        "arguments": datapoint["arguments"],
        "n_clauses": len(datapoint["clauses"]),
        "all_clauses": dict_clauses
    })

Secondo me ha poco senso salvare il dataset come dataframe dato che è praticamente solo testo. Comunque non dovrebbe essere difficilissimo tirarci fuori qualche statistica. Ho fatto degli esempi scemi qua:

In [4]:
n_arguments = []
n_premises = []
n_clauses = []
for case in refactored_dataset:
    n_arguments.append(len(case["arguments"]))
    n_clauses.append(case["n_clauses"])
    for argument in case["arguments"]:
        n_premises.append(len(argument["premises"]))
print(f"""
On average, a case has {np.mean(n_clauses):.2f} clauses with a median of {np.median(n_clauses):.0f} clauses per case.
On average, a case has {np.mean(n_arguments):.2f} arguments with a median of {np.median(n_arguments):.0f} arguments per case.
Each argument, on average, has: {np.mean(n_premises):.2f} premises with a median of {np.median(n_premises):.0f} premises per argument.
""")


On average, a case has 248.95 clauses with a median of 226 clauses per case.
On average, a case has 17.69 arguments with a median of 14 arguments per case.
Each argument, on average, has: 2.63 premises with a median of 2 premises per argument.



Qua puoi vedere come tirare fuori il testo di una conclusion o di una premise: ti basta usare la conclusion/premise come indice nel dizionario delle clause

In [5]:
premise = "\n\t".join([refactored_dataset[0]["all_clauses"][premise] for premise in refactored_dataset[0]["arguments"][0]["premises"]])
conclusion = refactored_dataset[0]["all_clauses"][refactored_dataset[0]["arguments"][0]["conclusion"]]
print(f"""
Here an example of an argument:
    - Premises:
        {premise}
    - Conclusion:
        {conclusion}
""")


Here an example of an argument:
    - Premises:
        The Commission notes that the applicant was detained after having been sentenced by the first instance court to 18 months' imprisonment.
	He was released after the Court of Appeal reviewed this sentence, reducing it to 15 months' imprisonment, convertible to a fine.
    - Conclusion:
        The Commission finds that the applicant was deprived of his liberty "after conviction by a competent court" within the meaning of Article 5 para. 1 (a) (Art. 5-1-a) of the Convention.



## Prepare for a task

### 6.1 Argument Clause Recognition

dato che non tutte le clause sono parte di una premise/conclusion, le dividiamo per cercare di tirare fuori un classificatore di clauses

In [6]:
ACR_x = []
ACR_y = []
for case in refactored_dataset:
    n_clauses = case["n_clauses"]
    clauses = case["all_clauses"]
    args_set = set()
    for argument in case["arguments"]:
        ACR_x.append(clauses[argument["conclusion"]])
        ACR_y.append(torch.tensor([1,0]))
        args_set.add(argument["conclusion"])
        for premise in argument["premises"]:
            ACR_x.append(clauses[premise])
            ACR_y.append(torch.tensor([1.,0]))
            args_set.add(premise)
    for clause_id in clauses.keys():
        if not clause_id in args_set:
            ACR_x.append(clauses[clause_id])
            ACR_y.append(torch.tensor([0,1.]))
print(f"""
There are:
      - {len(ACR_x)} clause
      - {len([a for a in ACR_y if a[0] == 1])} true clause
      - {len([a for a in ACR_y if a[0] == 0])} fake clause
""")


There are:
      - 10456 clause
      - 2694 true clause
      - 7762 fake clause



# Train

Da qui in avanti ti dovrebbe essere tutto abbastanza familiare dato che è praticamente lo stesso codice dell'assignment 2. Qua ho fatto solo 5 epoche e con un learning rate a caso ma è giusto per far vedere come funziona. Da ora si può iniziare a giocare...

In [69]:
tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

In [70]:
class Model(NeuralNetwork):
    def __init__(self, out_features:int, dropout:float = .3) -> None:
        super().__init__()
        self.encoder = RobertaModel.from_pretrained("FacebookAI/roberta-base")
        self.output_layer = nn.Linear(self.encoder.config.hidden_size, out_features)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input):
        _, encoded_input = self.encoder(**input, return_dict = False)
        encoded_input = self.dropout(encoded_input)
    
        return self.output_layer(encoded_input)
    
model = Model(2)

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
ACR_x_tokenized = dict_lists_to_list_of_dicts(tokenizer(ACR_x, padding=True, truncation=True, return_tensors='pt'))
train_dataloader, validation_dataloader, test_dataloader = get_dataloader(ACR_x_tokenized, ACR_y, 8, [9])

In [68]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("operating on device:", device)

operating on device: cuda:0


In [11]:
train_data, validation_data =   model.train_network(train_dataloader, 
                    validation_dataloader, 
                    torch.optim.SGD, 
                    loss_function=nn.CrossEntropyLoss(),
                    device=device, 
                    batch_size=32,
                    verbose=True, 
                    output_extraction_function= lambda x: torch.max(x, -1)[1].view(-1).cpu(), 
                    metrics={
                     "accuracy": accuracy_score, 
                     "f1_score": lambda y_true, y_pred: f1_score(y_true, y_pred, average="macro")},
                    learning_rate=1e-5,
                    epochs=5)

EPOCH 1 training loss:      0.576 - validation loss:      0.629                                                                                                                                                                                                                                                      
EPOCH 1 training accuracy:      0.747 - validation accuracy:      0.684
EPOCH 1 training f1_score:      0.478 - validation f1_score:      0.680
----------------------------------------------------------------------------------------------------

EPOCH 2 training loss:      0.568 - validation loss:      0.631                                                                                                                                                                                                                                                      
EPOCH 2 training accuracy:      0.746 - validation accuracy:      0.684
EPOCH 2 training f1_score:      0.477 - validation f1_score:  

## 6.2 Argument Relation Mining

In [32]:
def add_two_premises(idx_1, argument):
    idx_2 = randint(0, len(argument["premises"]) - 1)
    while idx_1 == idx_2:
       idx_2 = randint(0, len(argument["premises"]) - 1)
    premise_1 = argument["premises"][idx_1]
    premise_2 = argument["premises"][idx_2]
    return {
        "e1": clauses[premise_1],
        "e2": clauses[premise_2]
    }, (premise_1, premise_2)

def add_premise_conclusion(idx_1, argument):
    premise = argument["premises"][idx_1]
    conclusion = argument["conclusion"]
    if randint(0, 10) > 5: # with 50% chance we make the premise the first element
        return {
            "e1": clauses[premise],
            "e2": clauses[conclusion]
        }, (premise, conclusion)
    else: # with 50% chance we make the conclusion the first element
        return {
            "e2": clauses[conclusion],
            "e1": clauses[premise]
        }, (premise, conclusion)

In [65]:
ARM_x = []
ARM_y = []
FACTOR = .6
n_related_two_premises = 0
n_related_premise_conclusion = 0
n_unrelated_two_premises = 0
n_unrelated_two_conclusions = 0
n_unrelated_premise_conclusion = 0
for case in refactored_dataset: # for each case
    n_clauses = case["n_clauses"]
    clauses = case["all_clauses"]

    args_set = set() # get the number of clauses in arguments
    for argument in case["arguments"]: 
        args_set.add(argument["conclusion"])
        for premise in argument["premises"]:
            args_set.add(premise)

    n_related = int(len(args_set) * FACTOR) # get the number of elements we want from a given case 
    n_unrelated = int(len(args_set) * FACTOR)
    already_taken_couples = set()
    related = []
    unrelated = []
    while len(related) < n_related: # generate untill we need more related elements 
        while True:
            idx_argument = randint(0, len(case["arguments"]) - 1) # take a random argument
            argument = case["arguments"][idx_argument]
            idx_1 = randint(0, len(argument["premises"]) - 1) #take a random premise (we know we need at least 1 premise since there is only 1 conclusion)
            if len(argument["premises"]) > 1: # some arguments have only 1 premise
                two_premises = randint(0,10)
                if two_premises >= 4: # with 60% chance we get 2 random premises
                    element, (id_1, id_2) = add_two_premises(idx_1, argument)
                    if not (id_1 + id_2 in already_taken_couples or id_2 + id_1 in already_taken_couples): #we add them only if we do not already have selected the combination
                        related.append(element)
                        n_related_two_premises += 1
                        already_taken_couples.add(id_1 + id_2)
                        break
                else: #with 40% chance we get a premise and a conclusion
                    element, (id_1, id_2) = add_premise_conclusion(idx_1, argument)
                    if not (id_1 + id_2 in already_taken_couples or id_2 + id_1 in already_taken_couples): #only if the couple has not been already selected
                        related.append(element)
                        n_related_premise_conclusion += 1
                        already_taken_couples.add(id_1 + id_2)
                        break
            else: # if we have only one premise we just get the premise and conclusion
                element, (id_1, id_2) = add_premise_conclusion(idx_1, argument)
                if not (id_1 + id_2 in already_taken_couples or id_2 + id_1 in already_taken_couples):
                    related.append(element)
                    n_related_premise_conclusion += 1
                    already_taken_couples.add(id_1 + id_2)
                    break

    while len(unrelated) < n_unrelated: # generate untill we need more unrelated elements 
        while True:
            idx_argument_1 = randint(0, len(case["arguments"]) - 1) # select 2 random arguments 
            idx_argument_2 = randint(0, len(case["arguments"]) - 1)
            while idx_argument_1 == idx_argument_2: # they must be different
                idx_argument_2 = randint(0, len(case["arguments"]) - 1)
            argument_1 = case["arguments"][idx_argument_1]
            argument_2 = case["arguments"][idx_argument_2]
            type_relation = randint(0,9)
            if type_relation < 3: # with 30% chance we select two premises
                id_1 = randint(0, len(argument_1["premises"]) - 1)
                id_2 = randint(0, len(argument_2["premises"]) - 1)
                id_1 = argument_1["premises"][id_1]
                id_2 = argument_2["premises"][id_2]
                if not (id_1 + id_2 in already_taken_couples or id_2 + id_1 in already_taken_couples): # only if the couple is not already present in the dataset
                    unrelated.append({
                        "e2": clauses[id_1],
                        "e1": clauses[id_2]
                    })
                    n_unrelated_two_premises += 1
                    already_taken_couples.add(id_1 + id_2)
                    break
            elif type_relation < 6: # with 30% chance we select a premise and a conclusion
                prem = randint(0, 1) # we select the premise argument at random
                args = [argument_1, argument_2]
                id_1 = randint(0, len(args[prem]["premises"]) - 1)
                id_1 = args[prem]["premises"][id_1]
                id_2 = args[abs(prem - 1)]["conclusion"] # the other argument gives the conclusion
                if not (id_1 + id_2 in already_taken_couples or id_2 + id_1 in already_taken_couples): # only if the couple is not already present in the dataset
                    n_unrelated_premise_conclusion += 1
                    already_taken_couples.add(id_1 + id_2)
                    if randint(0, 5) > 5: # with 50% chance we put the premise first 
                        unrelated.append({
                            "e2": clauses[id_1],
                            "e1": clauses[id_2]
                        })
                    else: # with 50% chance we put the conclusion first 
                        unrelated.append({
                            "e2": clauses[id_2],
                            "e1": clauses[id_1]
                        })
                    break
            else: # with 30% chance we select 2 conclusions
                id_1 = argument_1["conclusion"]
                id_2 = argument_2["conclusion"]
                if not (id_1 + id_2 in already_taken_couples or id_2 + id_1 in already_taken_couples): # only if the couple is not already present in the dataset
                    unrelated.append({
                        "e2": clauses[id_1],
                        "e1": clauses[id_2]
                    })
                    n_unrelated_two_conclusions += 1
                    already_taken_couples.add(id_1 + id_2)
                    break

    for related_elem in related: # add all the new elements to the dataset
        ARM_x.append(related_elem)
        ARM_y.append(torch.tensor([1., 0]))
    for unrelated_elem in unrelated:
        ARM_x.append(unrelated_elem)
        ARM_y.append(torch.tensor([0, 1.]))
        
assert len([y for y in ARM_y if y[0] == 0]) == (n_unrelated_two_premises + n_unrelated_premise_conclusion + n_unrelated_two_conclusions)
assert len([y for y in ARM_y if y[0] == 1.]) == (n_related_premise_conclusion + n_related_two_premises) 
print(f"""
There are:
      - {len(ARM_x)} couples
      - {n_related_two_premises + n_related_premise_conclusion} related clauses:
        - {n_related_two_premises} are couple of clauses both premises on the same argument
        - {n_related_premise_conclusion} are couple of clauses premise and conclusion on the same argument
      - {n_unrelated_two_premises + n_unrelated_premise_conclusion + n_unrelated_two_conclusions} unrelated clauses:
        - {n_unrelated_two_premises} are couple with two premises from different arguments
        - {n_unrelated_premise_conclusion} are couple with a premise and a conclusion from two different arguments
        - {n_unrelated_two_conclusions} are couple with two conclusions from different arguments
""")


There are:
      - 3194 couples
      - 1597 related clauses:
        - 763 are couple of clauses both premises on the same argument
        - 834 are couple of clauses premise and conclusion on the same argument
      - 1597 unrelated clauses:
        - 518 are couple with two premises from different arguments
        - 468 are couple with a premise and a conclusion from two different arguments
        - 611 are couple with two conclusions from different arguments



### Train

Model1 prende le due componenti e le analizza indipendentemente per poi dare un output

Molel2 concatena componente 1 e componente 2 per analizzarle assieme

In [73]:
class Model1(NeuralNetwork):
    def __init__(self, out_features:int, dropout:float = .3) -> None:
        super().__init__()
        self.encoder = RobertaModel.from_pretrained("FacebookAI/roberta-base")
        self.output_layer = nn.Linear(self.encoder.config.hidden_size * 2, out_features)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input):
        _, encoded_input_1 = self.encoder(**input["e1"], return_dict = False)
        _, encoded_input_2 = self.encoder(**input["e2"], return_dict = False)
        encoded_input = self.dropout(torch.cat((encoded_input_1, encoded_input_2), dim=1))
    
        return self.output_layer(encoded_input)
    
class Model2(NeuralNetwork):
    def __init__(self, out_features:int, dropout:float = .3) -> None:
        super().__init__()
        self.encoder = RobertaModel.from_pretrained("FacebookAI/roberta-base")
        self.output_layer = nn.Linear(self.encoder.config.hidden_size, out_features)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input):
        input = {key: torch.cat((input["e1"][key], input["e2"][key]), dim=1) for key in input["e1"].keys()}
        _, encoded_input = self.encoder(**input, return_dict = False)
        encoded_input = self.dropout(encoded_input)
    
        return self.output_layer(encoded_input)

model1 = Model1(2)
model2 = Model2(2)

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [71]:
ARM_x_tokenized = dict_lists_to_list_of_dicts({
    key : dict_lists_to_list_of_dicts(tokenizer([v[key] for v in ARM_x], padding=True, truncation=True, return_tensors='pt'))
for key in ARM_x[0].keys()})
train_dataloader, validation_dataloader, test_dataloader = get_dataloader(ARM_x_tokenized, ARM_y, 8, [9])

In [74]:
train_data, validation_data =   model1.train_network(train_dataloader, 
                    validation_dataloader, 
                    torch.optim.SGD, 
                    loss_function=nn.CrossEntropyLoss(),
                    device=device, 
                    batch_size=32,
                    verbose=True, 
                    output_extraction_function= lambda x: torch.max(x, -1)[1].view(-1).cpu(), 
                    metrics={
                     "accuracy": accuracy_score, 
                     "f1_score": lambda y_true, y_pred: f1_score(y_true, y_pred, average="macro")},
                    learning_rate=1e-5,
                    epochs=5)

EPOCH 1 training loss:      0.696 - validation loss:      0.711                                                                                                                                                                                                                                                      
EPOCH 1 training accuracy:      0.500 - validation accuracy:      0.527
EPOCH 1 training f1_score:      0.328 - validation f1_score:      0.513
----------------------------------------------------------------------------------------------------

EPOCH 2 training loss:      0.696 - validation loss:      0.712                                                                                                                                                                                                                                                      
EPOCH 2 training accuracy:      0.501 - validation accuracy:      0.527
EPOCH 2 training f1_score:      0.324 - validation f1_score:  

In [75]:
train_data, validation_data =   model2.train_network(train_dataloader, 
                    validation_dataloader, 
                    torch.optim.SGD, 
                    loss_function=nn.CrossEntropyLoss(),
                    device=device, 
                    batch_size=32,
                    verbose=True, 
                    output_extraction_function= lambda x: torch.max(x, -1)[1].view(-1).cpu(), 
                    metrics={
                     "accuracy": accuracy_score, 
                     "f1_score": lambda y_true, y_pred: f1_score(y_true, y_pred, average="macro")},
                    learning_rate=1e-5,
                    epochs=5)

EPOCH 1 training loss:      0.699 - validation loss:      0.721                                                                                                                                                                                                                                                      
EPOCH 1 training accuracy:      0.500 - validation accuracy:      0.473
EPOCH 1 training f1_score:      0.323 - validation f1_score:      0.445
----------------------------------------------------------------------------------------------------

EPOCH 2 training loss:      0.697 - validation loss:      0.717                                                                                                                                                                                                                                                      
EPOCH 2 training accuracy:      0.500 - validation accuracy:      0.473
EPOCH 2 training f1_score:      0.324 - validation f1_score:  